In [1]:
import numpy as np
import pandas as pd
import re
import jsonlines
import os
import random

os.environ['KERAS_BACKEND'] = 'theano'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers.merge import concatenate

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from sklearn.metrics import accuracy_score

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [2]:
MAX_SEQUENCE_LENGTH = 50
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
# VALIDATION_SPLIT = 0.1111806
train_val_data =[]
truth_data = []
test_data = []
complete_data = []
count= 0
full_count=0
with jsonlines.open('instances.jsonl') as reader:
    for obj in reader.iter(type=dict, skip_invalid=True):
        count += 1
        full_count+=1
#         if (count > 17600):
#             test_data.append(obj)
#         if(count<=17600):
#             train_val_data.append(obj)
        complete_data.append(obj)
random.shuffle(complete_data)
complete_length = len(complete_data)
#complete_data = complete_data[random.sample(range(complete_length),complete_length)]
train_val_data = complete_data[:int(0.9*complete_length)]
test_data = complete_data[int(0.9*complete_length)+1:]
count = 0
truth_data = []
with jsonlines.open('truth.jsonl') as reader:
    for obj in reader.iter(type=dict, skip_invalid=True):
        truth_data.append(obj)

In [3]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"@", "", string)
    return string.lower()

In [4]:
def get_labels(vals_df):
    labels = []
    for i in vals_df.values:
        if(i[3]=="clickbait"):
            labels.append(1)
        else:
            labels.append(0)
    return labels

In [5]:
def get_title_df(vals):
    titles_df = []
    for i in range(len(vals)): ## For titles
        text = []
        k = vals[i][2]
        text.append(k)
        words = ""
        for string in text:
            string = clean_str(string)
            words +=" ".join(string.split())
        titles_df+=[words]
    return titles_df

In [6]:
def get_caption_df(vals):
    titles_df = []
    for i in range(len(vals)): ## For titles
        text = []
        k = vals[i][4]
        text.append(k)
        words = ""
        for string in text:
            string = clean_str(string)
            words +=" ".join(string.split())
        titles_df+=[words]
    return titles_df

In [7]:
def get_content_df(vals):
    content_df = []
    for i in range(len(vals)): ## For content
        text = []
        for j in range(2, 6):
            if(j==5):
                continue
            else:
                k = vals[i][j]
                if(j==6):
                    text.append(k)
                else:
                    text += (k)
        words = ""
        for string in text:
            string = clean_str(string)
            words += " ".join(string.split())
        content_df += [words]
    return content_df

In [8]:
def get_padded_sequences(df):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(df)
    sequences = tokenizer.texts_to_sequences(df)
    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    return data, word_index

In [9]:
print(type(train_val_data))

<class 'list'>


In [10]:
final_vals = []
data_df = pd.DataFrame.from_dict(train_val_data)
truth_data_df = pd.DataFrame.from_dict(truth_data)
train_captions_df = pd.read_csv('./total_captions.csv')
train_captions_df['id'] = train_captions_df['id'].apply(str)
# features = ["id", "postMedia", "postText", "targetCaptions", "targetParagraphs", "targetTitle", "targetKeywords",
#                 "targetDescription", "truthClass"]
train_1 = pd.merge(data_df, truth_data_df, on="id")
train = pd.merge(train_1,train_captions_df,on="id")
print(len(train))
features = ["id","postMedia","targetTitle",  "truthClass","caption"]
vals = train[features]
vals = vals.values.tolist()
for i in range(len(vals)):
    if vals[i][1] != []:
        final_vals.append([vals[i][0],vals[i][1],vals[i][2], vals[i][3], vals[i][4]])

9235


In [11]:
vals_df = pd.DataFrame(final_vals, columns=["id", "postMedia","targetTitle",  "truthClass","caption"])
print("Final vals length", len(final_vals))

Final vals length 9235


In [12]:
finalTestvals = []
test_data_df = pd.DataFrame.from_dict(test_data)
test_1 = pd.merge(test_data_df, truth_data_df, on="id")
test = pd.merge(test_1,train_captions_df,on="id")
test_vals = test[features].values.tolist()
for i in range(len(test_vals)):
    if test_vals[i][1] != []:
        finalTestvals.append([test_vals[i][0],test_vals[i][1],test_vals[i][2], test_vals[i][3], test_vals[i][4]])
test_vals_df = pd.DataFrame(finalTestvals, columns=["id", "postMedia","targetTitle",  "truthClass","caption"])
print("finalTestVals length", len(finalTestvals))

finalTestVals length 1070


In [13]:
labels = get_labels(vals_df)
tlabels = get_labels(test_vals_df)

In [14]:
print(test_vals_df)

                      id                                          postMedia  \
0     854801793351659520               [media/photo_854801791392915456.jpg]   
1     829062432131862528               [media/photo_829062429623582721.jpg]   
2     843507361704742912               [media/photo_843507359586639874.jpg]   
3     833442135621922817               [media/photo_833442132971180034.jpg]   
4     813942652374777856               [media/photo_813942650567008256.jpg]   
5     829144633544306689               [media/photo_829144630901870597.jpg]   
6     855293439218401280               [media/photo_855293433769885696.png]   
7     838569828357136384               [media/photo_838569825622360064.jpg]   
8     844586446430310400               [media/photo_844586443498573824.jpg]   
9     855533477810380800               [media/photo_855533475058917376.jpg]   
10    849397534770507776               [media/photo_849397456571944962.jpg]   
11    836208098167529472               [media/photo_

In [15]:
title_train = get_title_df(vals_df.values.tolist())
content_train = get_caption_df(vals_df.values.tolist())

title_train_df, t_word_index = get_padded_sequences(title_train)
content_train_df, c_word_index = get_padded_sequences(content_train)

Found 15788 unique tokens.
Found 483 unique tokens.


In [16]:
title_test = get_title_df(test_vals_df.values.tolist())
content_test = get_caption_df(test_vals_df.values.tolist())

title_test_df, title_test_index = get_padded_sequences(title_test)
content_test_df, content_test_index = get_padded_sequences(content_test)

Found 4575 unique tokens.
Found 310 unique tokens.


In [17]:
print(title_train)

['trump plays hardball with dems on obamacare payments', 'how emotion over pet care helps explain human health spending', 'brexit migration cap gets backing of former u k tory ministers', 'reviving this law would mean the end of megabanks quicktake q a', 'cbsn live streaming video news channel', 'charleston shooting survivors describe emotional pain dylann roof inflicted', 'dave chappelle makes rare public appearance to advocate for police reform', 'this real life giant walking robot is like something out of a sci fi movie', "comminsure deloitte report clears insurance arm of 'systemic' 'cultural' problems", 'animated map reveals the most popular pizza chains in every state', 'college dispenses morning after pill in vending machine', 'police officer ties tie for speeding student', 'from gandhi to guns an indian woman explores the nra convention', 'trump was caught practicing his speech so the internet made jokes', 'french election latest updates', 'lil wayne passes drake for most hot 1

In [18]:
print(content_train)

['a woman holding a cell phone in her hand', 'a brown dog is standing in the grass', 'a group of people standing around a building', 'a tall building with a clock on it', 'a man in a suit and a tie', 'a young boy is sitting on a bench', 'a man holding a tennis racket in his hand', 'a man riding a skateboard through a building', 'a close up of a person holding an umbrella', 'a video kite is flying in the air', 'a street sign with a sign on it', 'a couple of people standing next to each other', 'a woman holding a cell phone in her hand', 'a man standing in front of a tree', 'a group of different types of different colored kites', 'a man wearing a hat and a tie', 'a crowd of people standing in front of a crowd of people', 'a little boy is holding a baby', 'a man is holding a pair of scissors', 'a man in a hat is holding a camera', 'a train station with a lot of windows', 'a man riding skis down a snow covered slope', 'a couple of women standing next to each other', 'a group of people stan

In [19]:
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', title_train_df.shape)
print('Shape of label tensor:', labels.shape)
VALIDATION_SPLIT = 0.1111806
indices = np.arange(title_train_df.shape[0])
np.random.shuffle(indices)
data = title_train_df[indices]
content_data = content_train_df[indices]
# image_data = image_features[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

Shape of data tensor: (9235, 50)
Shape of label tensor: (9235, 2)


In [20]:
x_title_train = data[:-nb_validation_samples]
x_content_train = content_data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]

x_title_val = data[-nb_validation_samples:]
x_content_val = content_data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [21]:
x_title_test = title_test_df
x_content_test = content_test_df
y_test = tlabels

In [22]:
print('Training and validation sets')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

embeddings_index = {}
f=open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings_index))

Training and validation sets
[6432. 1777.]
[807. 219.]
Total 400000 word vectors.


In [23]:
t_embedding_matrix = np.random.random((len(t_word_index) + 1, EMBEDDING_DIM)) ##Titles
for word, i in t_word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        t_embedding_matrix[i] = embedding_vector

c_embedding_matrix = np.random.random((len(c_word_index) + 1, EMBEDDING_DIM)) ##Content
for word, i in c_word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        c_embedding_matrix[i] = embedding_vector

In [24]:
embedding_layer = Embedding(len(t_word_index) + 1, EMBEDDING_DIM, weights=[t_embedding_matrix], input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

content_embedding_layer = Embedding(len(c_word_index) + 1, EMBEDDING_DIM, weights=[c_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [25]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='float32')
content_data_input = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='float32')

embedded_sequences = embedding_layer(sequence_input)
print(embedded_sequences)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)

content_embedded_sequences = content_embedding_layer(content_data_input)
l_lstm_content = Bidirectional(LSTM(100))(content_embedded_sequences)

Reshape{3}.0


In [26]:
print(l_lstm_content)

Join.0


In [27]:
print("-----------------------------")
print(l_lstm.shape)

preds_title = Dense(2, activation='softmax')(l_lstm)

preds_content = Dense(2,activation='softmax')(l_lstm_content)

preds_add = concatenate([preds_title, preds_content], axis =-1)

preds = Dense(2)(preds_add)

model = Model([sequence_input, content_data_input], preds)
# model1.add_update(ac.AttentionWithContext()) ###############

checkpoint = ModelCheckpoint("weights-text-{epoch:02d}-{val_acc:.2f}.hdf5")
callbacks_list = [checkpoint]
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

-----------------------------
Shape.0


In [28]:
print("model fitting - Bidirectional LSTM with titles and content")
model.summary()
print('------')

model fitting - Bidirectional LSTM with titles and content
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 100)      1578900     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 50, 100)      48400       input_2[0][0]                    
__________________________________________________

In [29]:
 model.fit([x_title_train, x_content_train], y_train, validation_data=([x_title_val, x_content_val], y_val), epochs=10, batch_size=50, callbacks=callbacks_list)

Train on 8209 samples, validate on 1026 samples
Epoch 1/10
8209/8209 [==============================] - 36s 4ms/step - loss: 0.6963 - acc: 0.2526 - val_loss: 0.4219 - val_acc: 0.1745
Epoch 2/10
8209/8209 [==============================] - 36s 4ms/step - loss: 0.4063 - acc: 0.1826 - val_loss: 0.4046 - val_acc: 0.1696
Epoch 3/10
8209/8209 [==============================] - 37s 4ms/step - loss: 0.3744 - acc: 0.1641 - val_loss: 0.4040 - val_acc: 0.1481
Epoch 4/10
8209/8209 [==============================] - 37s 5ms/step - loss: 0.3415 - acc: 0.1480 - val_loss: 0.3810 - val_acc: 0.1433
Epoch 5/10
8209/8209 [==============================] - 38s 5ms/step - loss: 0.3264 - acc: 0.1329 - val_loss: 0.3949 - val_acc: 0.1530
Epoch 6/10
8209/8209 [==============================] - 37s 5ms/step - loss: 0.2924 - acc: 0.1180 - val_loss: 0.4058 - val_acc: 0.1550
Epoch 7/10
8209/8209 [==============================] - 38s 5ms/step - loss: 0.2583 - acc: 0.1018 - val_loss: 0.6792 - val_acc: 0.1598
Epoch 8

In [35]:
model.load_weights('weights-text-10-0.16.hdf5')

In [36]:
t_embedding_matrix = np.random.random((len(title_test_index) + 1, EMBEDDING_DIM)) ##Titles
for word, i in title_test_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        t_embedding_matrix[i] = embedding_vector

c_embedding_matrix = np.random.random((len(content_test_index) + 1, EMBEDDING_DIM)) ##Content
for word, i in content_test_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        c_embedding_matrix[i] = embedding_vector

In [37]:
embedding_layer = Embedding(len(title_test_index) + 1, EMBEDDING_DIM, weights=[t_embedding_matrix], input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

content_embedding_layer = Embedding(len(content_test_index) + 1, EMBEDDING_DIM, weights=[c_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [38]:
preds = model.predict([x_title_test, x_content_test], batch_size=50, verbose=1)

1070/1070 [==============================] - 1s 1ms/step


In [39]:
preds_new = []
for i in range(len(preds)):
    preds_new.append(preds[i][0] + preds[i][1])
print("Accuracy score on Test data ", accuracy_score(y_test, np.asarray(preds_new).round()))

Accuracy score on Test data  0.7953271028037383
